In [1]:
import numpy as np
import healpy as hp
from astropy.table import Table

In [2]:
NSIDE = 64
npix = hp.nside2npix(NSIDE)
total_degrees_in_sky = 4.*np.pi*(180./np.pi)**2
area_per_pixel = total_degrees_in_sky/(1.*npix)

In [3]:
def radec_to_sph(ra,dc):
    
    theta = (90.-dc)*np.pi/180.
    phi   = ra*np.pi/180.
    return theta, phi

In [4]:
def get_n(ra,dec,NSIDE):
    #Convert to ra/dec
    theta, phi = radec_to_sph(ra,dec)
    #set up the healpix grid.
    n = hp.ang2pix(NSIDE, theta, phi)    
    return n

### Median F-test catalog density

In [5]:
#First, let's get the median density of the F-test selected sources. 
Fcat = Table.read("../v1.1/Master_Catalog_F_test_AGN.fits")
Fcat2 = Table.read("../v1.1_complementary/Master_Catalog_F_test_AGN.complementary.fits")

In [6]:
#Join the coordinates of the two F-test catalogs.
k1 = len(Fcat['ra'])
k2 = len(Fcat2['ra'])
ra = np.zeros(k1+k2)
dec = np.zeros(k1+k2)
ra[:k1] = Fcat['ra']
ra[k1:] = Fcat2['ra']
dec[:k1] = Fcat['dec']
dec[k1:] = Fcat2['dec']

In [7]:
#Get the median field density
n_Ftot = get_n(ra, dec, NSIDE)
h_Ftot = np.histogram(n_Ftot,hp.nside2npix(NSIDE), range=(0,hp.nside2npix(NSIDE)-1))[0]
median_F_number = np.median(h_Ftot[h_Ftot>0])
median_F_density = median_F_number/area_per_pixel

In [8]:
print("The median density of F-test sources is {:.1f} per sq. deg.".format(median_F_density))
print("In regions that are not gaps or deeper fields, there are a total of {} sources".format(np.sum(h_Ftot[(h_Ftot>0.2*median_F_number) & (h_Ftot<3*median_F_number)])))

The median density of F-test sources is 97.7 per sq. deg.
In regions that are not gaps or deeper fields, there are a total of 1422505 sources


In [9]:
#Now, we need to define a gap density and an overdense field density to make the final catalog. To define the gap regions, we need to get the BIC densities. 
BIC_cat = Table.read("../v1.1/Master_Catalog_BIC_AGN_nb_le_6.fits")
BIC_cat2 = Table.read("../v1.1_complementary/Master_Catalog_BIC_AGN_nb_le_6.complementary.fits")

In [10]:
j1 = len(BIC_cat['ra'])
j2 = len(BIC_cat2['ra'])
ra_BIC = np.zeros(j1+j2)
dec_BIC = np.zeros(j1+j2)
ra_BIC[:j1] = BIC_cat['ra']
ra_BIC[j1:] = BIC_cat2['ra']
dec_BIC[:j1] = BIC_cat['dec']
dec_BIC[j1:] = BIC_cat2['dec']

In [11]:
n_BIC = get_n(ra_BIC, dec_BIC, NSIDE)
h_BIC = np.histogram(n_BIC,hp.nside2npix(NSIDE), range=(0,hp.nside2npix(NSIDE)-1))[0]

In [12]:
#Define the main survey, gaps and high density regions. 
main_survey_condition = (h_Ftot>=0.2*median_F_number) & (h_Ftot<=3*median_F_number)
gap_condition = (h_BIC>0) & (h_Ftot<0.2*median_F_number)
high_density_condition = h_Ftot>3*median_F_number

#Now, get the total number of sources that the final catalog will have if we keep the median number in all the cells of the main F-test survey region, half of that in the gaps and twice that in the high density regions. 
N_FR = len(h_Ftot[main_survey_condition])*median_F_number
N_GR = len(h_Ftot[gap_condition])*median_F_number*0.5
N_HR = len(h_Ftot[high_density_condition])*median_F_number*2
print(N_FR, N_GR, N_HR, N_FR+N_GR+N_HR)

1237790.0 240752.0 277324.0 1755866.0
